Беру лексему "лук". По словарю MAC берем значение 1 - "Огородное растение сем. лилейных, овощ". Значение 2 - "Ручное оружие для метания стрел в виде дуги, стянутой тетивой". Т.о. диагностические слова - овощ, зеленый; оружие, стрелять.
Контексты из НКРЯ для 1:  
а) Он мелко нарезал лук, морковь, чеснок, петрушку, салат ― все овощи, которые нашлись в холодильнике. 

б) На столе были разбросаны кучки зелёного лука и свежие огурцы.

Для 2:

а) Старшие стреляли птиц из самодельных луков.

б) Излюбленный вид оружия всадника ― лук.

In [0]:
context1a = 'Он мелко нарезал лук, морковь, чеснок, петрушку, салат ― все овощи, которые нашлись в холодильнике'
context1b = 'На столе были разбросаны кучки зелёного лука и свежие огурцы'
context2a = 'Старшие стреляли птиц из самодельных луков'
context2b = 'Излюбленный вид оружия всадника ― лук'

In [0]:
tolk1 = 'Огородное растение семейство лилейных, овощ'
tolk2 = 'Ручное оружие для метания стрел в виде дуги, стянутой тетивой'

In [0]:
!pip install pymorphy2

In [0]:
!pip install pymystem3==0.1.10

In [0]:
!pip install allennlp

In [0]:
!pip install git+https://github.com/lopuhin/python-adagram.git

In [0]:
!wget https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib

In [0]:
import adagram
from allennlp.commands.elmo import ElmoEmbedder
import pandas as pd
from lxml import html
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
# from tqdm.notebook import tqdm
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.cluster import *
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
# stops = set(stopwords.words('russian'))

def normalize_pm(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

In [0]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [10]:
vm.word_sense_probs('вечер')

[(0, 0.13080149644282904),
 (1, 0.21073024350994726),
 (2, 0.2956921249405029),
 (3, 0.14701596077005963),
 (4, 0.21575999977552957)]

In [11]:
vm.word_sense_probs('лук')

[(0, 0.27467387333364635),
 (1, 0.17510064513645113),
 (2, 0.25182447428150506),
 (3, 0.0852764349918363),
 (4, 0.2131232975158767)]

In [12]:
vm.sense_neighbors('лук', 0)

[('морковь', 0, 0.8814602),
 ('помидор', 0, 0.879848),
 ('лук-порей', 0, 0.8786377),
 ('зелень', 0, 0.872901),
 ('нарезать', 0, 0.8665899),
 ('нашинковывать', 0, 0.86500126),
 ('сельдерей', 0, 0.86482704),
 ('петрушка', 0, 0.8591249),
 ('мелко', 0, 0.8526903),
 ('нарубать', 0, 0.85126823)]

In [13]:
vm.sense_neighbors('лук', 1)

[('арбалет', 0, 0.7783164),
 ('дротик', 0, 0.6703703),
 ('пращ', 0, 0.6575549),
 ('самострел', 0, 0.64090955),
 ('мушкет', 0, 0.60409844),
 ('аркебуза', 0, 0.5913013),
 ('стрелять', 0, 0.5830755),
 ('дробовик', 0, 0.5742717),
 ('пневматический', 1, 0.5695134),
 ('аркебуз', 0, 0.56936765)]

In [24]:
vm.sense_neighbors('лук', 3)

[('лука', 1, 0.6011757),
 ('евангелист', 0, 0.54460436),
 ('иаков', 0, 0.5291651),
 ('лука', 2, 0.5226612),
 ('матфей', 1, 0.52241987),
 ('матфей', 0, 0.5201959),
 ('деяние', 1, 0.50425464),
 ('иоанн', 2, 0.50399673),
 ('апостол', 1, 0.49078336),
 ('ириней', 1, 0.49005806)]

In [26]:
vm.sense_neighbors('лук', 4)

[('лука', 0, 0.805669),
 ('арбалет', 1, 0.78230816),
 ('колчан', 0, 0.7731788),
 ('тетива', 0, 0.72229147),
 ('стрела', 0, 0.6901984),
 ('копье', 0, 0.68338805),
 ('самострел', 0, 0.67639554),
 ('копье', 1, 0.6679108),
 ('секира', 0, 0.6648324),
 ('палица', 0, 0.6603637)]

In [14]:
mystem = Mystem()

def disambiguate(model, word, context):
    word, _ = lemmatized_context(word)
    probs = model.disambiguate(word, lemmatized_context(context))
    return 1 + probs.argmax()


def lemmatized_context(s):
    return [w.lower() for w in mystem.lemmatize(" ".join(tokenize(s)))]

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


In [18]:
print ('Значение для контекстов значения "овощ" : ' + str (disambiguate(vm, 'лук', context1a)) + ', ' +str (disambiguate(vm, 'лук', context1b)))

Значение для контекстов значения "овощ" : 1, 1


In [20]:
print ('Значение для контекстов значения "оружие" : ' + str (disambiguate(vm, 'лук', context2a)) + ', ' +str (disambiguate(vm, 'лук', context2b)))

Значение для контекстов значения "оружие" : 2, 2


Ну, кажется, то что нужно!

In [28]:
print ('Значение для толкования значения "овощ" : ' + str (disambiguate(vm, 'лук', tolk1)) + '\nЗначение для толкования значения "оружие" : ' +str (disambiguate(vm, 'лук', tolk2)))

Значение для толкования значения "овощ" : 1
Значение для толкования значения "оружие" : 5


Первое толкование правильно, а второе нет. По выдаче adagram видно, что слово "оружие" не является ключевым, однако контекст2b был обработан правильно. 

Пункт 4.

In [0]:
!wget https://github.com/sjut/HSE-Compling/raw/master/seminars/data/relations_with_concepts.csv

In [0]:
import csv

rels_list = []
with open("relations_with_concepts.csv", newline='', encoding='utf8') as rels:
    reader = csv.DictReader(rels, delimiter="\t")
    for row in reader:
        rels_list.append(row)


def get_supc2(concept_list, rels_list, has_up=True, depth=0, max_depth=-1):
    """
    Get list of all hypernym chains of the query
    - up a level
    - add all 'выше' concepts to list
    [[level_1, level_2.1, level_3.1], [level_1, level_2.2, level_3.2], etc...]

    :param concept_list: search input
    :param rels_list: imported set of relations
    :param max_depth: maximum allowed number of hypernyms
    :param has_up: (internal) bool(current top concept has a superconcept)
    :param depth: (internal) current depth in the ontology
    :return: list of superconcept for every meaning of query
    """
    new_cl = concept_list[:]
    if (not has_up) or depth >= max_depth > 0:
        return new_cl
    has_up = False
    for chain in concept_list:
        index = new_cl.index(chain)
        word = chain[-1]
        for row in rels_list:
            new_chain = chain[:]
            if row['from'].lower() == word.lower() and row['relation'] == 'ВЫШЕ':
                new_chain.append(row['to'].lower())
                new_cl.insert(index + 1, new_chain)
                has_up = True
        if has_up:
            new_cl.remove(chain)
    return get_supc2(new_cl, rels_list, has_up, depth+1, max_depth)


def get_supc(concept_list, rels_list, has_up=True, depth=0, max_depth=-1):
    """
    Find list of all hypernyms of query by level down
    [[level_1], [level_2.1, level_2.2], [level_3.1, level_3.2, level_3.3], etc...]

    :param concept_list: search input
    :param rels_list: imported set of relations
    :param max_depth: maximum allowed number of hyponyms
    :param has_up: (internal) bool(current top concept has a subconcept)
    :param depth: (internal) current depth in the ontology
    :return: list of subconcepts for every meaning of query
    """
    if (not has_up) or depth >= max_depth > 0:
        return concept_list
    has_up = False
    new_list = []
    for word in concept_list[-1]:
        for row in rels_list:
            if row['from'].lower() == word.lower() and row['relation'] == 'ВЫШЕ':
                if all(row['to'].lower() not in hypo for hypo in concept_list):
                    new_list.append(row['to'].lower())
                    has_up = True
    if has_up:
        concept_list.append(new_list)
    return get_supc(concept_list, rels_list, has_up, depth + 1, max_depth)


def get_subc2(concept_list, rels_list, has_down=True, depth=0, max_depth=-1):
    """
    Get list of all hyponym chains for word in query
    - down a level
    - add all 'ниже' concepts to list
    [[level_1, level_2.1, level_3.1], [level_1, level_2.2, level_3.2], etc...]

    :param concept_list: search input
    :param rels_list: imported set of relations
    :param max_depth: maximum allowed number of hyponyms
    :param has_down: (internal) bool(current top concept has a subconcept)
    :param depth: (internal) current depth in the ontology
    :return: list of subconcepts for every meaning of query
    """
    new_cl = concept_list[:]
    if (not has_down) or depth >= max_depth > 0:
        return new_cl
    for chain in concept_list:
        has_down = False
        index = new_cl.index(chain)
        word = chain[-1]
        for row in rels_list:
            new_chain = chain[:]
            if row['from'].lower() == word.lower() and row['relation'] == 'НИЖЕ':
                new_chain.append(row['to'].lower())
                new_cl.insert(index + 1, new_chain)
                has_down = True
        if has_down:
            new_cl.remove(chain)
    return get_subc2(new_cl, rels_list, has_down, depth+1, max_depth)


def get_subc(concept_list, rels_list, has_down=True, depth=0, max_depth=-1):
    """
    Find list of all hyponyms of query by level down
    [[level_1], [level_2.1, level_2.2], [level_3.1, level_3.2, level_3.3], etc...]

    :param concept_list: search input
    :param rels_list: imported set of relations
    :param max_depth: maximum allowed number of hyponyms
    :param has_down: (internal) bool(current top concept has a subconcept)
    :param depth: (internal) current depth in the ontology
    :return: list of subconcepts for every meaning of query
    """
    if (not has_down) or depth >= max_depth > 0:
        return concept_list
    has_down = False
    new_list = []
    for word in concept_list[-1]:
        for row in rels_list:
            if row['from'].lower() == word.lower() and row['relation'] == 'НИЖЕ':
                if all(row['to'].lower() not in hypo for hypo in concept_list):
                    new_list.append(row['to'].lower())
                    has_down = True
    if has_down:
        concept_list.append(new_list)
    return get_subc(concept_list, rels_list, has_down, depth+1, max_depth)

In [43]:
get_subc([['лук']], rels_list)

[['лук']]

In [0]:
get_subc([['растение']], rels_list)

In [42]:
get_subc2([['луковичное растение']], rels_list)

[['луковичное растение', 'чеснок'],
 ['луковичное растение', 'лук (растение)', 'зеленый лук'],
 ['луковичное растение', 'лук (растение)', 'черемша'],
 ['луковичное растение', 'лук (растение)', 'лук-порей'],
 ['луковичное растение', 'лук (растение)', 'репчатый лук'],
 ['луковичное растение', 'крокус'],
 ['луковичное растение', 'подснежник'],
 ['луковичное растение', 'нарцисс'],
 ['луковичное растение', 'гиацинт (цветок)'],
 ['луковичное растение', 'тюльпан']]

Если честно, я не совсем поняла п.4 из задания. Поэтому предложу метод, как мне кажется, о котором идет речь.

1) Мы берем контекст с нужным словом

2) Adagram выдает нам значение, в котором используется данное слово в этом предложении

3) Мы смотрим на слова-соседи (так же с помощью adagram)

4) Осуществляем поиск по тезариусу по нашему слову и ищем в его тезаурус_соседях слова_соседи_adagram

5) Выявляем общее, смотрим, где эти соседи расположены относительно нашего слова (например, нас интересуют слова "выше": овощи, оружие, но не интересуют слова "ниже": лук-репка).

6) Таким образом можно составить толкование (или соотнести с имеющимися, не знаю что нужно в задании).

Пункт 5.

Я опять немного запуталась в задании(( Но если я правильно поняла, accuracy для контекстов 100%, а для толкований 50%.